# 오차역전파법
* 오차역전파법(backpropagation) : 가중치 매개변수의 기울기를 효율적으로 계산한다.
* 수식
* 계산그래프(시각적 이해)

## 계산 그래프
* 계산 그래프(computational graph) : 계산 과정을 그래프로 나타낸다.  
* 노드 node + 에지 edge(노드 사이 직선) 구성

### 계산 그래프로 풀다
1. 현빈 군이 슈퍼에서 1개에 100원인 사과를 2개 샀다. 지불 금액은? 소비세 10% 부과된다.  
2. 현빈 군이 슈퍼에서 사과 2개, 귤 3개 샀다. 사과는 1개 100원, 귤은 1개 150원이고, 소비세 10%이다. 지불 금액은?  

    * 순전파(forward propagation) : 계산을 왼쪽에서 오른쪽으로 진행한다.  
    * 역전파(backward propagation) : 반대방향으로 계산한다.

### 국소적 계산
* 계산 그래프 특징 : 국소적 계산을 전파하여 최종 결과를 얻는다.  
* 국소적 : 자신과 직접 관계된 작은 범위  
* 국소적 계산 : 전체와 관련 없이 자신과 관계된 정보만 결과로 출력한다.(local, not global). 

![img](./deep_learning_images/fig_5-4.png)

각 노드는 자신과 관련한 계산(두 숫자의 덧셈) 외에 아무것도 하지 않는다.  
계산 그래프는 국소적 계산에 집중한다. 각 노드의 국소적 계산 결과를 전달하여 전체를 구성한다.

### 왜 계산 그래프로 푸는가?
* 계산 그래프를 사용하는 가장 큰 이유  
역전파를 통해 **미분**을 효율적으로 계산한다.  
![img](./deep_learning_images/fig_5-5.png)

## 연쇄법칙
연쇄법칙(chain rule) : 국소적 미분을 전달하는 원리
### 계산 그래프의 역전파
$$y = f(x)$$
![img](./deep_learning_images/fig_5-6.png)
### 연쇄법칙이란?
* 합성 함수 : 여러 함수로 구성된 함수  
* 연쇄법칙은 합성 함수의 미분에 대한 성질  
_합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다._  

$${\partial z \over \partial x} = {\partial z \over \partial t} {\partial t \over \partial x}$$
* 예시
$$ z = t^2 \\ t = x + y$$
### 연쇄법칙과 계산 그래프
* 연쇄법칙 계산을 계산 그래프로 나타낸다. 2제곱 계산은 **2로 나타낸다.
![img](./deep_learning_images/fig_5-7.png)  

* 역전파가 하는 일은 연쇄 법칙의 원리와 같다.  
* 그림에 국소적 미분(편미분) 값을 대입하면 다음과 같다.
![img](./deep_learning_images/fig_5-8.png)

## 역전파
### 덧셈 노드의 역전파
### 곱셈 노드의 역전파
### 사과 쇼핑의 예

## 단순한 계층 구현하기
### 곱셈 계층

In [2]:
class MulLayer:
    def __init__(self):#x, y 초기화
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x, y 바꾸기
        dy = dout * self.x
        
        return dx, dy

In [4]:
#사과 쇼핑 구현하기

apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [5]:
# 각 변수의 미분은 backward()
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


![img](./deep_learning_images/fig_5-16.png)

In [6]:
# 덧셈 계층
class AddLayer:
    def __init__(self):
        pass# 초기화 필요 없어서 아무것도 하지 말라는 명령이다!
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

![img](./deep_learning_images/fig_5-17.png)

In [7]:
# 사과 2개, 오렌지 3개 구매
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer() # 덧셈 노드
mul_tax_layer = MulLayer() # 곱셈 노드

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)

dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange_num, dorange, dtax)

715.0000000000001
110.00000000000001 2.2 165.0 3.3000000000000003 650


## 활성화 함수 계층 구현하기
### ReLU 계층
![img](./deep_learning_images/fig_5-18.png)

In [9]:
class Relu:
    def __init__(self):
        self.mask = None # mask : True/False 구성된 넘파이 배열
        
    def forward(self, x):
        self.mask = (x <= 0) # 0보다 작은 인덱스 True, 0보다 크면 False 유지
        out = x.copy() #x 값을 복제한다. 원본 파일 변경을 방지한다.
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [11]:
import numpy as np
#mask 알아보기
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [12]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


### Sigmoid 계층
$$ y = {1 \over {1 + \exp(-x)}}$$
* 계산 그래프(순전파)
![img](./deep_learning_images/fig_5-19.png) 

계산 그래프(역전파)의 단계별 흐름

In [14]:
# 시그모이드 계층
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1+np.exp(-x))
        self.out = out
        
        return out
    
    def backward(Self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## Affine/Softmax 계층 구현하기
### Affine 계층

In [16]:
X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

print(X.shape)
print(W.shape)
print(B.shape)

Y = np.dot(X, W) + B

(2,)
(2, 3)
(3,)


* 어파인 변환(affine transformation) : 신경망의 순전파 때 수행하는 행렬의 곱
* 어파인 계층 : 어파인 변환을 수행하는 처리

### 배치용 Affine 계층

In [17]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [18]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [19]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [22]:
dB = np.sum(dY, axis = 0) # 편향의 역전파는 그 두 데이터에 대한 미분을 데이터마다 더해서 구현한다.
dB

array([5, 7, 9])

In [24]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(Self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

### Softmax with Loss 계층
출력층에서 사용하는 소프트맥스 함수

In [25]:
#Softmax with loss 구현

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss =cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

## 오차역전파법 구현하기
### 신경망 학습의 전체 그림
### 오차역전파법을 적용한 신경망 구현하기
TwoLayerNet 클래스 인스턴스 변수
1. params
    * 딕셔너리 변수, 신경망의 매개변수를 보관
    


In [28]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
            return x
        
        #x 입력 데이터, t 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1 : t = np.argmax(t, axis=1) # 파이썬 기초문법 참고
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과를 저장한다.
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads # 기울기가 반환된다.
    

### 오차역전파법으로 구한 기울기 검증하기
* 기울기 확인(gradient check)

In [29]:
import sys, os
sys.path.append(os.pardir) #부모디렉터리 설정
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize= True, one_hot_label= True)

network = TwoLayerNet(input_size = 784, hidden_size= 50, output_size= 10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치 차이의 절댓값을 구하고, 그 절댓값의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:2.08360027012189e-10
b1:1.0034358459090229e-09
W2:7.089479214632267e-08
b2:1.417670533840343e-07


* 수치 미분과 오차역전파법으로 구한 기울기의 차이가 매우 작은 것을 알 수 있다.  
(= 정확한 기울기를 구한 것이다.)

### 오차역전파법을 사용한 학습 구현하기

In [34]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# data load
(x_train, t_train), (x_test, t_test) = load_mnist(normalize= True, one_hot_label=True)
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1) # 에폭당 반복

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기 구하기
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("훈련 정확도 : " + str(train_acc))
        print("시험 정확도: " + str(test_acc))




훈련 정확도 : 0.10218333333333333
시험 정확도: 0.101
훈련 정확도 : 0.7876333333333333
시험 정확도: 0.7922
훈련 정확도 : 0.87525
시험 정확도: 0.8785
훈련 정확도 : 0.89895
시험 정확도: 0.8996
훈련 정확도 : 0.9072333333333333
시험 정확도: 0.908
훈련 정확도 : 0.9135833333333333
시험 정확도: 0.9151
훈련 정확도 : 0.9190833333333334
시험 정확도: 0.9197
훈련 정확도 : 0.9226666666666666
시험 정확도: 0.9232
훈련 정확도 : 0.9266333333333333
시험 정확도: 0.9282
훈련 정확도 : 0.9307
시험 정확도: 0.9305
훈련 정확도 : 0.9325166666666667
시험 정확도: 0.933
훈련 정확도 : 0.9357
시험 정확도: 0.9362
훈련 정확도 : 0.93775
시험 정확도: 0.938
훈련 정확도 : 0.9399666666666666
시험 정확도: 0.9388
훈련 정확도 : 0.94225
시험 정확도: 0.9408
훈련 정확도 : 0.9449833333333333
시험 정확도: 0.9427
훈련 정확도 : 0.94585
시험 정확도: 0.9434


## 정리

* 이번 장에서 배운 내용
    * 계산 그래프를 이용하면 계산 과정을 시각적으로 파악 가능
    * 계산 그래프의 노드는 국소적 계산으로 구성한다. 국소적 계산을 조합해서 전체 계산을 구한다.
    * 계산 그래프의 순전파는 일반적인 계산이고, 역전파는 각 노드의 미분을 구한다.
    * 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산한다.(오차역전파법).
    * 수치 미분과 오차역전파법의 결과를 비교하여 구현에 잘못이 없는지 확인 가능(기울기 확인).